# import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import cv2
import math
import pandas as pd
import numpy as np

# 情報取得

In [ ]:
# 大会ごとのファイルをまとめる
df_2018 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/2023年度スポコン柔道/世界選手権_略WC_CSV/csv_2018WC.csv", encoding="shift-jis")
df_2019 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/2023年度スポコン柔道/世界選手権_略WC_CSV/csv_2019WC.csv", encoding="shift-jis")
df_2021 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/2023年度スポコン柔道/世界選手権_略WC_CSV/csv_2021WC.csv", encoding="shift-jis")
df_all = pd.concat([df_2018, df_2019, df_2021])
df_all.to_csv("/content/drive/MyDrive/Colab Notebooks/2023年度スポコン柔道/2018-2019-2021WC.csv")

In [ ]:
# csvファイルからstopMillis情報を取得する
#pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/2023年度スポコン柔道/2018-2019-2021WC.csv", columns=['年','大会','試合種別','階級','白選手','白国','青選手','青国','映像分','映像秒','EVENT_NAME','PLAY_SNAME'])
df = pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/2023年度スポコン柔道/2018-2019-2021WC.csv", index_col=0)
df = df[['年','大会','試合種別','階級','白選手','白国','青選手','青国','映像分','映像秒','EVENT_NAME','PLAY_SNAME']]
# フィルタリング 一本を取り出す
filter = df.EVENT_NAME.isin(['一本'])
print(filter)

0       False
1       False
2       False
3       False
4       False
        ...  
3121    False
3122    False
3123    False
3124    False
3125    False
Name: EVENT_NAME, Length: 10201, dtype: bool


In [ ]:
# csvファイルからstopMillis情報を取得する
df = pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/2023年度スポコン柔道/2018-2019-2021WC.csv", index_col=0)
df = df[['年','大会','試合種別','階級','白選手','白国','青選手','青国','映像分','映像秒','EVENT_NAME','PLAY_SNAME']]

# フィルタリング 一本を取り出す
filter = df.EVENT_NAME.isin(['一本'])
df= df[filter].reset_index(drop = True)
# df

# フィルタリング 欲しい年を取り出す
filter = df.年.isin([2021])
df= df[filter].reset_index(drop = True)
# df

# フィルタリング 投技を取り出す
nagewaza_text = "一本背負投,踵返,肩車,小内返,朽木倒,双手刈,帯落,帯取返,背負投,背負落,隅落,掬投,体落,内股すかし,浮落,山嵐,跳腰,払腰,腰車,大腰,袖釣込腰,釣腰,釣込腰,浮腰,後腰,移腰,足車,出足払,膝車,跳腰返,払腰返,払釣込足,小外刈,小外掛,小内刈,送足払,大外返,大外車,大車,大外刈,大外落,大内刈,大内返,支釣込足,燕返,内股,内股返,引込返,隅返,巴投,俵返,裏投,抱分,跳巻込,払巻込,大外巻込,外巻込,谷落,内巻込,内股巻込,小内巻込,浮技,横掛,横車,横落,横分,蟹挟,河津掛"
nagewaza = nagewaza_text.split(",") # カンマで分割してリストに格納
filter = df.PLAY_SNAME.isin(nagewaza)
df= df[filter].reset_index(drop = True)
df

# フィルタリング 固技を取り出す
# katamewaza_text = "肩固,袈裟固,崩袈裟固,後袈裟固,崩上四方固,上四方固,縦四方固,横四方固,裏固,浮固,逆十字絞,裸絞,片羽絞,片十字絞,片手絞,並十字絞,送襟絞,両手絞,袖車絞,突込絞,三角絞,胴絞,腕挫脚固,腕挫腹固,腕挫膝固,腕挫十字固,腕挫三角固,腕挫手固,腕挫腋固,腕挫腕固,腕緘,足緘"
# katamewaza = katamewaza_text.split(",")
# filter = df.PLAY_SNAME.isin(katamewaza)
# df= df[filter].reset_index(drop = True)
# df

# Nanを抜く
df.fillna('NaN',inplace=True) # NaNをstringへ変わる
df.drop(df[df['青国'].str.contains(pat='NaN',regex=False)==True].index,inplace=True)
df.drop(df[df['白国'].str.contains(pat='NaN',regex=False)==True].index,inplace=True)
df = df.reset_index(drop=True)
df

,年,大会,試合種別,階級,白選手,白国,青選手,青国,映像分,映像秒,EVENT_NAME,PLAY_SNAME
0,2021,WCB,P1,60,LEUNG Hon Man,HKG,LOPES Rodrigo Costa,POR,3,11,一本,大外落
1,2021,WCB,P1,60,AKKUS Mihrac,TUR,DE BARROS E SILVA Alexandre,CPV,2,39,一本,大外巻込
2,2021,WCB,P1,60,RAMOS Jose,GUA,KOKOLAYEV Matan,ISR,10,22,一本,小外掛
3,2021,WCB,P1,60,MILIC Nemanja,SRB,LESIUK Artem,UKR,3,47,一本,払巻込
4,2021,WCB,P1,60,BOUDA Romaric Wend-Yam,FRA,MAMMADSOY Davud,AZE,10,24,一本,小内刈
...,...,...,...,...,...,...,...,...,...,...,...,...
158,2021,WCB,PF,P78,MOJICA Melissa,PUR,ASAHINA Sarah,JPN,4,43,一本,大外刈
159,2021,WCB,PF,P78,ALTHEMAN Maria Suelen,BRA,TOMITA Wakaba,JPN,4,56,一本,体落
160,2021,WCB,RF,P78,TOLOFUA Julia,FRA,MOJICA Melissa,PUR,2,58,一本,大外刈
161,2021,WCB,SF,P78,TOMITA Wakaba,JPN,SOUZA Beatriz,BRA,6,33,一本,袖釣込腰


In [ ]:
startMillis = []
stopMillis = []
Video_length = 3 # 単位は秒

for i in range(len(df)):
  stopmillisecond = ( df['映像分'][i] * 60 + df['映像秒'][i] ) * 1000
  startmillisecond = stopmillisecond - 1000 * Video_length
  stopMillis.append(stopmillisecond)
  startMillis.append(startmillisecond)

# targetFileName, destFileNameを作る
targetFileName = []
destFileName = []
for i in range(len(df)):
  target_name = str(df['年'][i]) + str(df['大会'][i]) + '_' + str(df['階級'][i]) + '_' + str(df['試合種別'][i]) + '_' + str(df['白選手'][i]) + '_' + str(df['白国'][i]) + '_' + str(df['青選手'][i]) + '_' + str(df['青国'][i]) + '.mp4'
  # target_name = str(df['年'][i]) + str(df['大会'][i]) + '_' + str(df['階級'][i]) + '_' + str(df['試合種別'][i]) + '_' + str(df['白選手'][i]) + '_' + str(df['白国'][i]) + '_' + str(df['青選手'][i]) + '_' + str(df['青国'][i])
  # target_name = str(df['年'][i]) + ' ' + str(df['階級'][i]) + ' ' + str(df['試合種別'][i]) + ' ' + str(df['白選手'][i]) + ' ' + str(df['青選手'][i])
  dest_name = str(df['年'][i]) + str(df['大会'][i]) + '_' + str(df['階級'][i]) + '_' + str(df['試合種別'][i]) + '_' + str(df['白選手'][i]) + '_' + str(df['白国'][i]) + '_' + str(df['青選手'][i]) + '_' + str(df['青国'][i]) + '_1.mp4'
  targetFileName.append(f"{target_name}")
  destFileName.append(f"{dest_name}")

print(stopMillis)
print(startMillis)
print(targetFileName)
print(destFileName)

[191000, 159000, 622000, 227000, 624000, 182000, 298000, 133000, 333000, 273000, 148000, 77000, 107000, 163000, 106000, 270000, 316000, 359000, 161000, 296000, 255000, 268000, 440000, 89000, 139000, 367000, 492000, 462000, 75000, 82000, 140000, 167000, 163000, 25000, 220000, 368000, 193000, 367000, 375000, 417000, 354000, 288000, 435000, 125000, 24000, 170000, 204000, 228000, 151000, 355000, 144000, 110000, 131000, 149000, 672000, 142000, 177000, 384000, 130000, 486000, 294000, 256000, 182000, 422000, 61000, 361000, 407000, 159000, 143000, 29000, 461000, 580000, 272000, 22000, 122000, 197000, 277000, 304000, 21000, 318000, 112000, 336000, 118000, 171000, 382000, 243000, 148000, 118000, 23000, 81000, 51000, 91000, 265000, 142000, 394000, 217000, 132000, 148000, 235000, 515000, 180000, 590000, 283000, 555000, 145000, 261000, 16000, 67000, 467000, 400000, 88000, 20000, 70000, 199000, 45000, 262000, 268000, 137000, 405000, 165000, 300000, 399000, 182000, 195000, 370000, 197000, 95000, 6400

In [ ]:
df['startMillis'] = startMillis
df['stopMillis'] = stopMillis
df['targetFileName'] = targetFileName
df['destFileName'] = destFileName
df

,年,大会,試合種別,階級,白選手,白国,青選手,青国,映像分,映像秒,EVENT_NAME,PLAY_SNAME,startMillis,stopMillis,targetFileName,destFileName
0,2021,WCB,P1,60,LEUNG Hon Man,HKG,LOPES Rodrigo Costa,POR,3,11,一本,大外落,188000,191000,2021WCB_60_P1_LEUNG Hon Man_HKG_LOPES Rodrigo ...,2021WCB_60_P1_LEUNG Hon Man_HKG_LOPES Rodrigo ...
1,2021,WCB,P1,60,AKKUS Mihrac,TUR,DE BARROS E SILVA Alexandre,CPV,2,39,一本,大外巻込,156000,159000,2021WCB_60_P1_AKKUS Mihrac_TUR_DE BARROS E SIL...,2021WCB_60_P1_AKKUS Mihrac_TUR_DE BARROS E SIL...
2,2021,WCB,P1,60,RAMOS Jose,GUA,KOKOLAYEV Matan,ISR,10,22,一本,小外掛,619000,622000,2021WCB_60_P1_RAMOS Jose_GUA_KOKOLAYEV Matan_I...,2021WCB_60_P1_RAMOS Jose_GUA_KOKOLAYEV Matan_I...
3,2021,WCB,P1,60,MILIC Nemanja,SRB,LESIUK Artem,UKR,3,47,一本,払巻込,224000,227000,2021WCB_60_P1_MILIC Nemanja_SRB_LESIUK Artem_U...,2021WCB_60_P1_MILIC Nemanja_SRB_LESIUK Artem_U...
4,2021,WCB,P1,60,BOUDA Romaric Wend-Yam,FRA,MAMMADSOY Davud,AZE,10,24,一本,小内刈,621000,624000,2021WCB_60_P1_BOUDA Romaric Wend-Yam_FRA_MAMMA...,2021WCB_60_P1_BOUDA Romaric Wend-Yam_FRA_MAMMA...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,2021,WCB,PF,P78,MOJICA Melissa,PUR,ASAHINA Sarah,JPN,4,43,一本,大外刈,280000,283000,2021WCB_P78_PF_MOJICA Melissa_PUR_ASAHINA Sara...,2021WCB_P78_PF_MOJICA Melissa_PUR_ASAHINA Sara...
159,2021,WCB,PF,P78,ALTHEMAN Maria Suelen,BRA,TOMITA Wakaba,JPN,4,56,一本,体落,293000,296000,2021WCB_P78_PF_ALTHEMAN Maria Suelen_BRA_TOMIT...,2021WCB_P78_PF_ALTHEMAN Maria Suelen_BRA_TOMIT...
160,2021,WCB,RF,P78,TOLOFUA Julia,FRA,MOJICA Melissa,PUR,2,58,一本,大外刈,175000,178000,2021WCB_P78_RF_TOLOFUA Julia_FRA_MOJICA Meliss...,2021WCB_P78_RF_TOLOFUA Julia_FRA_MOJICA Meliss...
161,2021,WCB,SF,P78,TOMITA Wakaba,JPN,SOUZA Beatriz,BRA,6,33,一本,袖釣込腰,390000,393000,2021WCB_P78_SF_TOMITA Wakaba_JPN_SOUZA Beatriz...,2021WCB_P78_SF_TOMITA Wakaba_JPN_SOUZA Beatriz...


In [ ]:
# データフレームからランダムに100行を抽出
random_NagewazaFileName100 = df.sample(n=100, random_state=1)  # random_stateを指定することで再現性を確保

# 結果を表示
random_NagewazaFileName100.sort_index()

,年,大会,試合種別,階級,白選手,白国,青選手,青国,映像分,映像秒,EVENT_NAME,PLAY_SNAME,startMillis,stopMillis,targetFileName,destFileName
0,2021,WCB,P1,60,LEUNG Hon Man,HKG,LOPES Rodrigo Costa,POR,3,11,一本,大外落,188000,191000,2021WCB_60_P1_LEUNG Hon Man_HKG_LOPES Rodrigo ...,2021WCB_60_P1_LEUNG Hon Man_HKG_LOPES Rodrigo ...
2,2021,WCB,P1,60,RAMOS Jose,GUA,KOKOLAYEV Matan,ISR,10,22,一本,小外掛,619000,622000,2021WCB_60_P1_RAMOS Jose_GUA_KOKOLAYEV Matan_I...,2021WCB_60_P1_RAMOS Jose_GUA_KOKOLAYEV Matan_I...
4,2021,WCB,P1,60,BOUDA Romaric Wend-Yam,FRA,MAMMADSOY Davud,AZE,10,24,一本,小内刈,621000,624000,2021WCB_60_P1_BOUDA Romaric Wend-Yam_FRA_MAMMA...,2021WCB_60_P1_BOUDA Romaric Wend-Yam_FRA_MAMMA...
5,2021,WCB,P1,60,LEE Harim,KOR,ALDHUFAIRI Omar,KUW,3,2,一本,背負投,179000,182000,2021WCB_60_P1_LEE Harim_KOR_ALDHUFAIRI Omar_KU...,2021WCB_60_P1_LEE Harim_KOR_ALDHUFAIRI Omar_KU...
6,2021,WCB,P2,60,BASSOU Issam,MAR,CALLE Dilmer,PER,4,58,一本,大内刈,295000,298000,2021WCB_60_P2_BASSOU Issam_MAR_CALLE Dilmer_PE...,2021WCB_60_P2_BASSOU Issam_MAR_CALLE Dilmer_PE...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,2021,WCB,P2,P78,NUNES Rochele,POR,TOMITA Wakaba,JPN,4,53,一本,隅落,290000,293000,2021WCB_P78_P2_NUNES Rochele_POR_TOMITA Wakaba...,2021WCB_P78_P2_NUNES Rochele_POR_TOMITA Wakaba...
157,2021,WCB,P2,P78,IBRAHIM Maryan,KSA,KORO Essohanam Noeline,TOG,0,37,一本,小外掛,34000,37000,2021WCB_P78_P2_IBRAHIM Maryan_KSA_KORO Essohan...,2021WCB_P78_P2_IBRAHIM Maryan_KSA_KORO Essohan...
158,2021,WCB,PF,P78,MOJICA Melissa,PUR,ASAHINA Sarah,JPN,4,43,一本,大外刈,280000,283000,2021WCB_P78_PF_MOJICA Melissa_PUR_ASAHINA Sara...,2021WCB_P78_PF_MOJICA Melissa_PUR_ASAHINA Sara...
160,2021,WCB,RF,P78,TOLOFUA Julia,FRA,MOJICA Melissa,PUR,2,58,一本,大外刈,175000,178000,2021WCB_P78_RF_TOLOFUA Julia_FRA_MOJICA Meliss...,2021WCB_P78_RF_TOLOFUA Julia_FRA_MOJICA Meliss...


# dropboxでサーチするキーワード作成
（ビデオnameを「 OR 」でつなぐ）

In [ ]:
keyword = " OR ".join(df['targetFileName'][0:30])
print(keyword)

2021WCB_60_P1_LEUNG Hon Man_HKG_LOPES Rodrigo Costa_POR.mp4 OR 2021WCB_60_P1_AKKUS Mihrac_TUR_DE BARROS E SILVA Alexandre_CPV.mp4 OR 2021WCB_60_P1_RAMOS Jose_GUA_KOKOLAYEV Matan_ISR.mp4 OR 2021WCB_60_P1_MILIC Nemanja_SRB_LESIUK Artem_UKR.mp4 OR 2021WCB_60_P1_BOUDA Romaric Wend-Yam_FRA_MAMMADSOY Davud_AZE.mp4 OR 2021WCB_60_P1_LEE Harim_KOR_ALDHUFAIRI Omar_KUW.mp4 OR 2021WCB_60_P2_BASSOU Issam_MAR_CALLE Dilmer_PER.mp4 OR 2021WCB_60_P2_LKHAGVAJAMTS Unubold_MGL_DIAZ Adonis_USA.mp4 OR 2021WCB_60_P2_KOKOLAYEV Matan_ISR_KOGA Genki_JPN.mp4 OR 2021WCB_60_P2_SHAMSHADIN Magzhan_KAZ_LESIUK Artem_UKR.mp4 OR 2021WCB_60_P3_PRECIADO Lenin_ECU_LKHAGVAJAMTS Unubold_MGL.mp4 OR 2021WCB_60_PF_KYRGYZBAYEV Gusman_KAZ_LKHAGVAJAMTS Unubold_MGL.mp4 OR 2021WCB_48_P1_CSERNOVICZKI Eva_HUN_MURATBAEVA Gulnur_UZB.mp4 OR 2021WCB_48_P1_VARGAS LEY Mary Dee_CHI_LIKMABAM Shushila Devi_IND.mp4 OR 2021WCB_48_P1_MAMIRA Luca_HUN_GALBADRAKH Otgontsetseg_KAZ.mp4 OR 2021WCB_48_P1_CHAKIR Aziza_MAR_SIDEROT Maria_POR.mp4 OR 2021WCB

In [ ]:
for i in range(48):
  from_num = 15* i
  to_num = 15 * (i+1)
  keyword = " OR ".join(df['targetFileName'][from_num:to_num])
  length = len(df['targetFileName'][from_num:to_num])
  print(f'from{from_num}to{to_num}')
  print(keyword)
  i += 1

from0to15
2021WCB_60_P1_LEUNG Hon Man_HKG_LOPES Rodrigo Costa_POR.mp4 OR 2021WCB_60_P1_AKKUS Mihrac_TUR_DE BARROS E SILVA Alexandre_CPV.mp4 OR 2021WCB_60_P1_RAMOS Jose_GUA_KOKOLAYEV Matan_ISR.mp4 OR 2021WCB_60_P1_MILIC Nemanja_SRB_LESIUK Artem_UKR.mp4 OR 2021WCB_60_P1_BOUDA Romaric Wend-Yam_FRA_MAMMADSOY Davud_AZE.mp4 OR 2021WCB_60_P1_LEE Harim_KOR_ALDHUFAIRI Omar_KUW.mp4 OR 2021WCB_60_P2_BASSOU Issam_MAR_CALLE Dilmer_PER.mp4 OR 2021WCB_60_P2_LKHAGVAJAMTS Unubold_MGL_DIAZ Adonis_USA.mp4 OR 2021WCB_60_P2_KOKOLAYEV Matan_ISR_KOGA Genki_JPN.mp4 OR 2021WCB_60_P2_SHAMSHADIN Magzhan_KAZ_LESIUK Artem_UKR.mp4 OR 2021WCB_60_P3_PRECIADO Lenin_ECU_LKHAGVAJAMTS Unubold_MGL.mp4 OR 2021WCB_60_PF_KYRGYZBAYEV Gusman_KAZ_LKHAGVAJAMTS Unubold_MGL.mp4 OR 2021WCB_48_P1_CSERNOVICZKI Eva_HUN_MURATBAEVA Gulnur_UZB.mp4 OR 2021WCB_48_P1_VARGAS LEY Mary Dee_CHI_LIKMABAM Shushila Devi_IND.mp4 OR 2021WCB_48_P1_MAMIRA Luca_HUN_GALBADRAKH Otgontsetseg_KAZ.mp4
from15to30
2021WCB_48_P1_CHAKIR Aziza_MAR_SIDEROT Maria_

# unzip

In [ ]:
import zipfile

# ジップファイルのパスを指定
zip_file_path = "/content/drive/MyDrive/Colab Notebooks/2023年度スポコン柔道/データ/投技zipファイル/Dropbox-2021.zip"  # ここにジップファイルの名前を指定

# 解凍先のディレクトリを指定
extracted_folder = "/content/drive/MyDrive/Colab Notebooks/sports/sports_competition/データ/2021WC"  # ここに解凍先のディレクトリ名を指定

# ジップファイルを解凍
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder)


In [ ]:
# ジップファイルのパスを指定
zip_file_path = "/content/drive/MyDrive/Colab Notebooks/sports/sports_competition/データ/Dropbox-2019.zip"  # ここにジップファイルの名前を指定

# 解凍先のディレクトリを指定
extracted_folder = "/content/drive/MyDrive/Colab Notebooks/sports/sports_competition/データ/2019WC"  # ここに解凍先のディレクトリ名を指定

# ジップファイルを解凍
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder)

In [ ]:
# ジップファイルのパスを指定
zip_file_path = "/content/drive/MyDrive/Colab Notebooks/sports/sports_competition/データ/Dropbox.zip"  # ここにジップファイルの名前を指定

# 解凍先のディレクトリを指定
extracted_folder = "/content/drive/MyDrive/Colab Notebooks/sports/sports_competition/データ/2018WC"  # ここに解凍先のディレクトリ名を指定

# ジップファイルを解凍
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder)

# ダウンロードしたファイルの名前を確認

In [ ]:
# フォルダのファイル名前を読み取る
import os

# 文件夹路径
folder_path = '/content/drive/MyDrive/Colab Notebooks/2023年度スポコン柔道/データ/2021WC'

# 获取文件夹中的所有文件名
file_names = os.listdir(folder_path)

# # 打印文件名
# for file_name in file_names:
#     print(file_name)


In [ ]:
file_names += file_names

In [ ]:
print(len(file_names))
print(file_names)

178
['2021WCB_81_P2_GOTONOAGA Dorin_MDA_BARTO Alex_SVK.mp4', '2021WCB_100_P3_ADAMIAN Arman__DVARBY Joakim_SWE.mp4', '2021WCB_60_P1_RAMOS Jose_GUA_KOKOLAYEV Matan_ISR.mp4', '2021WCB_66_P3_YONDONPERENLEI Baskhuu_MGL_CHOPANOV Murad_.mp4', '2021WCB_73_P2_SMAGULOV Zhansay_KAZ_MAKHMADBEKOV Makhmadbek_.mp4', '2021WCB_48_P2_KHUBULOVA Irena__ALVAREZ Luz_COL.mp4', '2021WCB_81_F_CASSE Matthias_BEL_GRIGALASHVILI Tato_GEO.mp4', '2021WCB_78_P1_NIKIFOROVA Niurguiana__TURCHYN Anastasiya_UKR.mp4', '2021WCB_81_P3_BOLTABOEV Sharofiddin_UZB_TCKAEV Zelim_AZE.mp4', '2021WCB_P100_P2_ALLERSTORFER Daniel_AUT_MARINIC Enej_SLO.mp4', '2021WCB_66_RF_SHAMILOV Yakub__MINKOU Dzmitry_BLR.mp4', '2021WCB_P100_PF_KAGEURA Kokoro_JPN_PUUMALAINEN Martti_FIN.mp4', '2021WCB_100_P2_IMALA Otto_EST_KUMRIC Zlatko_CRO.mp4', '2021WCB_81_P4_BOLTABOEV Sharofiddin_UZB_MAISURADZE Luka_GEO.mp4', '2021WCB_81_P1_JAMAL Tareq__LUCENTI Emmanuel_ARG.mp4', '2021WCB_66_P2_CADET Gerard_HAI_CHOPANOV Murad_.mp4', '2021WCB_70_P1_GERCSAK Szabina_HUN

In [ ]:
file_names_new = file_names.sort()
print(file_names)

['2018WCB_100_3M_DARWISH Ramadan_EGY_ILYASOV Niyaz_RUS.mp4', '2018WCB_100_P1_BAUZA Karolis_LTU_MEDINA Lewis_DOM.mp4', '2018WCB_100_P1_DUBRETA Anto_MNE_LI Huilin_CHN.mp4', '2018WCB_100_P1_GASIMOV Elmar_AZE_PACEK Martin_SWE.mp4', '2018WCB_100_P1_KALIVOGUI Michel_GUI_SVIRYD Mikita_BLR.mp4', '2018WCB_100_P1_KOSTER Jason_NZL_REYES Kyle_CAN.mp4', '2018WCB_100_P1_KUMRIC Zlatko_CRO_MARET Cyrille_FRA.mp4', '2018WCB_100_P1_SAVYTSKIY Anton_UKR_SHAH Hussain Shah_PAK.mp4', '2018WCB_100_P1_TAKAYAWA Tevita_FIJ_DOSEN Bojan_SRB.mp4', '2018WCB_100_P1_WOLF Aaron_JPN_HAJAS Matej_SVK.mp4', '2018WCB_100_P2_CHO Guham_KOR_MARET Cyrille_FRA.mp4', '2018WCB_100_P2_FONSECA Jorge_POR_HORAK Michal_CZE.mp4', '2018WCB_100_P2_KHURRAMOV Mukhammadkarim_UZB_REYES Kyle_CAN.mp4', '2018WCB_100_P2_KORREL Michael_NED_DVARBY Joakim_SWE.mp4', '2018WCB_100_P2_PALTCHIK Peter_ISR_BORODAVKO Jevgenijs_LAT.mp4', '2018WCB_100_P2_PANTIC Danilo_MNE_GASIMOV Elmar_AZE.mp4', '2018WCB_100_P2_SAVYTSKIY Anton_UKR_MUKETE Daniel_BLR.mp4', '2018

In [ ]:
# df.to_excel('/content/drive/MyDrive/Colab Notebooks/sports/sports_competition/データ/2018-2019-2021-DownloadedFile.xlsx', sheet_name='DownloadedFile')

# 動画からカットする

In [ ]:
print(file_names[246])

2019WCT_81_P3_SANTOS Eduardo Yudy_BRA_BRIAND Etienne_CAN.mp4


In [ ]:
index = df.index[df['targetFileName'] == file_names[246]]
print(index)

Int64Index([171], dtype='int64')


In [ ]:
for i in range(len(file_names)):
  if file_names[i] == '2019WCT_52_P2_PEREZ BOX Ana_ESP_DELGADO Angelica_USA.mp4':
    print(i)
  else:
    i += 1

245


In [ ]:
for i in range(len(file_names)):
# index２４６はエラーなので、２４７から
# for i in range(246, len(file_names)):
  element_to_find = file_names[i]
  index = df.index[df['targetFileName'] == element_to_find]
  if len(index) == 0:
    i += 1
  else:
    index = index.values
    index = int(index)
    print(index)

54
2
70
59
143
151
121
63
101
146
84
99
103
133
162
149
108
120
153
60
42
8
118
147
158
148
26
128
109
92
124
161
142
37
129
35
135
19
38
9
79
21
77
85
6
72
95
3
156
113
160
15
115
41
61
116
102
67
55
105
34
47
145
86
123
134
5
22
98
141
97
96
46
25
125
50
122
53
131
36
39
83
45
56
117
27
51
48
132
130
30
155
139
10
68
31
32
12
7
4
81
43
20
91
76
74
89
104
58
14
24
144
71
112
62
82
159
94
111
154
140
136
126
11
52
18
75
127
93
16
150
57
17
64
114
138
29
100
28
23
137
0
65
110
87
78
33
107
73
80
69
13
1
157
90
88
44
106


In [ ]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/sports/sports_competition/データ/2021WC/')

In [ ]:
import cv2
import math
import pandas as pd

# 動画を開始ミリ秒～終了ミリ秒までカットする
def trimVideo(targetFileName, destFileName, startMillis, stopMillis):
    # FPS、フレーム数・開始終了フレーム番号取得
    videoCapture = cv2.VideoCapture(targetFileName)
    fps = videoCapture.get(cv2.CAP_PROP_FPS)
    totalFrames = videoCapture.get(cv2.CAP_PROP_FRAME_COUNT)
    startFrameIndex = math.ceil(fps * startMillis / 1000)
    stopFrameIndex = math.ceil(fps * stopMillis / 1000)
    if startFrameIndex < 0:
        startFrameIndex = 0
    if stopFrameIndex >= totalFrames:
        stopFrameIndex = totalFrames - 1
    videoCapture.set(cv2.CAP_PROP_POS_FRAMES, startFrameIndex)
    frameIndex = startFrameIndex

    # 開始～終了地点までフレーム分割
    imgArr = []
    while frameIndex <= stopFrameIndex:
        _, img = videoCapture.read()
        imgArr.append(img)
        frameIndex += 1

    # 分割フレームをmp4動画に再構成
    fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
    video = None
    tmpVideoFileName = destFileName
    for img in imgArr:
        if video is None:
            h, w, _ = img.shape
            video = cv2.VideoWriter(tmpVideoFileName, fourcc, 20.0, (w, h))
        video.write(img)
    video.release()

if __name__ == "__main__":
  count = 0
  for i in range(len(file_names)):
  # for i in range(247, len(file_names)):
    element_to_find = file_names[i]
    index = df.index[df['targetFileName'] == element_to_find]
    if len(index) == 0:
      i += 1
    else:
      index = index.values
      index = int(index)
      i += 1
      startMillis = df['startMillis'][index]
      stopMillis = df['stopMillis'][index]
      targetFileName = df['targetFileName'][index]
      destFileName = df['destFileName'][index]

      trimVideo(targetFileName, destFileName, startMillis, stopMillis)
      count += 1

  print(count)



158
